In [2]:
%%html
<!-- This cell makes the font bigger to make it easy to read. Adjust to taste -->
<style>
.cell, .CodeMirror pre{ 
    font-size: 150%;
    line-height: 125%;
}
</style>

# COSC470 Assignment 2, 2018

## Name: Hannah Clark-Younger
## Due Date: Monday September 24th

For assignment 2 you need to implement machine learning algorithm(s) to label faces according to:
- sex (male/female)
- age (child/teen/adult/senior)
- expression (smiling/serious)

A data set from MIT is made available, along with code to read the images and labels into `numpy` arrays. 
These arrays are divided into training, validation, and testing data sets.

You may use any machine learning algorithms you like to classify the faces.
Techniques you may find useful that we've looked at include:
- Decision trees and random forests
- Boosting (and AdaBoost in particular)
- Support Vector Machines (SVMs)
- Face detection (to focus on the key parts of the image)
- EigenFaces (for dimensionality reduction)
- Neural networks in TensorFlow
- CNNs in TensorFlow

## Submission Requirements

You should submit a version of this Notebook renamed to `YourName.ipynb`, so my submission would be `StevenMills.ipynb`. 
You can assume that the same libraries that are available in the COSC470 Anaconda environment on the lab machines are available.
In particular, you can use numpy, scipy, OpenCV, and TensorFlow.

I should be able to open your Notebook and run it. The Notebook should contain the code to construct and train your classifier(s) from the training data (using the validation data appropriately) and then to compute the labels of the training data through a call to `computeLabels`, which has a stub implementation at the end of this notebook. 

## Marking Scheme

A rough marking scheme is given below. This is intentionally fairly open, so that I can give you marks for doing good stuff without having to predetermine what stuff is good.

- 10 marks for the discussion of choice of algorithms and training strategy
- 10 marks for the explanation and clear implementation
- 5 marks for performance

### Algorithm Choice and Training

I will be looking for a description of the algorithm(s) chosen, why you chose that approach, and how you developed, trained and evaluated your method.
You should think about issues such as how to best make use of the training and validation data and how to select parameters for your chosen method.

You are not restricted to a single classifier or method. If you find it useful to determine age labels first and then use that to help determine expression, then that is fine. If you want to use an SVM for sex classification, but a boosted classifier for age, that's also fine.
However, you should discuss why you chose to use the methods you have chosen.

### Explanation and Clear Implementation

You should implement your chosen algorithm(s) using the training and validation data sets provided. 
Jupyter notebooks let you interleave discussion and code, so you should clearly describe how your implementation works.
You can include mathematics if needed using \\(\LaTeX\\)-style markup as demonstrated in the lecture notebooks.
I'll be looking for clear implementations that illustrate good practice in training and evaluation.

It is expected that you will make appropriate use of libraries such as OpenCV and TensorFlow where appropriate, but your explanation should your understanding of these tools clear. 
For example, if you choose to use a convolutional network, you should explain your architecture, how it relates to the code, and give some justification for the various parameters that you need to select when making a CNN.

### Performance

The last cell of the notebook has a function that takes a face data set and produces labels as a result.
You should modify this so that it uses your machine learning algorithms to generate the labels.
I will then use these labels to compare your results to the ground truth.
I may also shuffle the training, validation, and testing data sets around before running your code.

# The Data Set

The following code reads the data into training, testing, and validation sets.
It assumes that the `.zip` of labelled face data set from the course website has been unzipped into the same directory as the notebook.
There are 1997 training images, and 998 each test and training images.

In [1]:
import numpy as np

# Read in training data and labels

# Some useful parsing functions

# male/female -> 0/1
def parseSexLabel(string):
    if (string.startswith('male')):
        return 0
    if (string.startswith('female')):
        return 1
    print("ERROR parsing sex from " + string)


# child/teen/adult/senior -> 0/1/2/3
def parseAgeLabel(string):
    if (string.startswith('child')):
        return 0
    if (string.startswith('teen')):
        return 1
    if (string.startswith('adult')):
        return 2
    if (string.startswith('senior')):
        return 3
    print("ERROR parsing age from " + string)


# serious/smiling -> 0/1
def parseExpLabel(string):
    if (string.startswith('serious')):
        return 0
    if (string.startswith('smiling') or string.startswith('funny')):
        return 1
    print("ERROR parsing expression from " + string)


# Count number of training instances

numTraining = 0

for line in open("MITFaces/faceDR"):
    if line.find('_missing descriptor') < 0:
        numTraining += 1

dimensions = 128 * 128

trainingFaces = np.zeros([numTraining, dimensions])
trainingSexLabels = np.zeros(numTraining)  # Sex - 0 = male; 1 = female
trainingAgeLabels = np.zeros(numTraining)  # Age - 0 = child; 1 = teen; 2 = male
trainingExpLabels = np.zeros(numTraining)  # Expression - 0 = serious; 1 = smiling

index = 0
for line in open("MITFaces/faceDR"):
    if line.find('_missing descriptor') >= 0:
        continue
    # Parse the label data
    parts = line.split()
    trainingSexLabels[index] = parseSexLabel(parts[2])
    trainingAgeLabels[index] = parseAgeLabel(parts[4])
    trainingExpLabels[index] = parseExpLabel(parts[8])
    # Read in the face
    fileName = "MITFaces/rawdata/" + parts[0]
    fileIn = open(fileName, 'rb')
    trainingFaces[index, :] = np.fromfile(fileIn, dtype=np.uint8, count=dimensions) / 255.0
    fileIn.close()
    # And move along
    index += 1

# Count number of validation/testing instances

numValidation = 0
numTesting = 0

# Assume they're all Validation
for line in open("MITFaces/faceDS"):
    if line.find('_missing descriptor') < 0:
        numTraining += 1
    numValidation += 1

# And make half of them testing
numTesting = int(numValidation / 2)
numValidation -= numTesting

validationFaces = np.zeros([numValidation, dimensions])
validationSexLabels = np.zeros(numValidation)  # Sex - 0 = male; 1 = female
validationAgeLabels = np.zeros(numValidation)  # Age - 0 = child; 1 = teen; 2 = male
validationExpLabels = np.zeros(numValidation)  # Expression - 0 = serious; 1 = smiling

testingFaces = np.zeros([numTesting, dimensions])
testingSexLabels = np.zeros(numTesting)  # Sex - 0 = male; 1 = female
testingAgeLabels = np.zeros(numTesting)  # Age - 0 = child; 1 = teen; 2 = male
testingExpLabels = np.zeros(numTesting)  # Expression - 0 = serious; 1 = smiling

index = 0
for line in open("MITFaces/faceDS"):
    if line.find('_missing descriptor') >= 0:
        continue

    # Parse the label data
    parts = line.split()
    if (index < numTesting):
        testingSexLabels[index] = parseSexLabel(parts[2])
        testingAgeLabels[index] = parseAgeLabel(parts[4])
        testingExpLabels[index] = parseExpLabel(parts[8])
        # Read in the face
        fileName = "MITFaces/rawdata/" + parts[0]
        fileIn = open(fileName, 'rb')
        testingFaces[index, :] = np.fromfile(fileIn, dtype=np.uint8, count=dimensions) / 255.0
        fileIn.close()
    else:
        vIndex = index - numTesting
        validationSexLabels[vIndex] = parseSexLabel(parts[2])
        validationAgeLabels[vIndex] = parseAgeLabel(parts[4])
        validationExpLabels[vIndex] = parseExpLabel(parts[8])
        # Read in the face
        fileName = "MITFaces/rawdata/" + parts[0]
        fileIn = open(fileName, 'rb')
        validationFaces[vIndex, :] = np.fromfile(fileIn, dtype=np.uint8, count=dimensions) / 255.0
        fileIn.close()

    # And move along
    index += 1
print("Data loaded")

Data loaded


# CONVNET FOR FACE CLASSIFICATION



In [ ]:
import tensorflow as tf
import cv2
import scipy
import random

####### MODIFIABLE PARAMETERS ######   

task = "Sex" # Options are "Sex", "Age", "Expression"

batch_size = 16
n_epochs = 1000

n_filters_conv1 = 64
filter_size_conv1 = 3
n_filters_conv2 = 128
filter_size_conv2 = 3
n_filters_conv3 = 256
filter_size_conv3 = 3
fc_layer_size = 512

display_step = 1
saver_step = 10

####################################

def make_one_hot(labels):
    global n_classes
    one_label = np.zeros(n_classes)
    new_labels = [one_label]*len(labels)
    for i in range(len(labels)):
        #print(i)
        one_label = np.zeros(n_classes)
        one_label[int(labels[i])] = 1
        new_labels[i] = one_label
        #print(labels[i])
        #print(new_labels[i])
    return np.array(new_labels)

class Dataset:
    def __init__(self,data,labels):
        #print(data.shape)
        self.data = data.reshape([-1,128,128,1]) #tf.convert_to_tensor(data.reshape([-1,128,128,1]), dtype=tf.float32)
        self.labels = labels #.reshape(-1,n_classes) # n_classes
        self.batch_index = 0
        
    def randomize(self, sess):
        shuffled_data = np.empty(self.data.shape, dtype=self.data.dtype)
        shuffled_labels = np.empty(self.labels.shape, dtype=self.labels.dtype)
        permutation = np.random.permutation(len(self.data))
        for old_index, new_index in enumerate(permutation):
            shuffled_data[new_index] = self.data[old_index]
            shuffled_labels[new_index] = self.labels[old_index]
        self.data = shuffled_data
        self.labels = shuffled_labels
    
    def next_batch(self, b_size):
        start = self.batch_index
        end = self.batch_index + b_size
        self.batch_index = end
        return self.data[start:end], self.labels[start:end]
    
def conv_pool_relu_layer(input, n_input, n_filters, filter_size):  
    weights = tf.Variable(tf.truncated_normal(shape=[filter_size, filter_size, n_input, n_filters], stddev=0.05))
    biases = tf.Variable(tf.constant(0.05, shape=[n_filters]))
    conv_layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')
    conv_layer += biases
    c_m_layer = tf.nn.max_pool(value=conv_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    c_m_r_layer = tf.nn.relu(c_m_layer)
    return c_m_r_layer

def flat_layer(input_layer):
    shape = input_layer.get_shape()
    num_features = shape[1:4].num_elements()
    flat_layer = tf.reshape(input_layer, [-1, num_features])
    return flat_layer

def fc_layer(input, n_inputs, n_outputs, use_relu=True):
    weights = tf.Variable(tf.truncated_normal(shape=[n_inputs, n_outputs], stddev=0.05))
    biases = tf.Variable(tf.constant(0.05, shape=[n_outputs]))
    fc_layer = tf.matmul(input, weights) + biases
    if use_relu:
        fc_layer = tf.nn.relu(fc_layer)
    return fc_layer

#print("after layer defns, before model defined")

if task == "Sex":
    n_classes = 2
    train_labels = make_one_hot(trainingSexLabels)
    valid_labels = make_one_hot(validationSexLabels)
    test_labels = make_one_hot(testingSexLabels)
    train_data = Dataset(trainingFaces,train_labels)
    valid_data = Dataset(validationFaces,valid_labels)
    test_data = Dataset(testingFaces,test_labels)
    model = "sex-model"
elif task == "Age":
    n_classes = 4
    train_labels = make_one_hot(trainingAgeLabels)
    valid_labels = make_one_hot(validationAgeLabels)
    test_labels = make_one_hot(testingAgeLabels)
    train_data = Dataset(trainingFaces,train_labels)
    valid_data = Dataset(validationFaces,valid_labels)
    test_data = Dataset(testingFaces,test_labels)
    model = "age-model"
elif task == "Expression":
    n_classes = 2 
    train_labels = make_one_hot(trainingExpLabels)
    valid_labels = make_one_hot(validationExpLabels)
    test_labels = make_one_hot(testingExpLabels)
    train_data = Dataset(trainingFaces,train_labels)
    valid_data = Dataset(validationFaces,valid_labels)
    test_data = Dataset(testingFaces,test_labels)
    model = "exp-model"
else:
    print("Please set task to one of the three options")

img_size = 128
num_channels = 1 # greyscale (I think)
n_batches = trainingFaces.shape[0]//batch_size

with tf.device('/cpu:0'):
    # set up basic (change to GoogLeNet?) model
    g = tf.Graph()
    with g.as_default():
        X = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels], name='X')
        #print(X.shape)
        y_true = tf.placeholder(tf.float32, shape=[None,n_classes], name='y_true') # None, 1 OR n_classes
        y_true_class = tf.argmax(y_true, dimension=1)
        conv1 = conv_pool_relu_layer(input=X, n_input=num_channels, n_filters=n_filters_conv1, filter_size=filter_size_conv1)
        conv2 = conv_pool_relu_layer(input=conv1, n_input=n_filters_conv1, n_filters=n_filters_conv2, filter_size=filter_size_conv2) 
        conv3 = conv_pool_relu_layer(input=conv2, n_input=n_filters_conv2, n_filters=n_filters_conv3,filter_size=filter_size_conv3)        
        flat = flat_layer(conv3)
        fc1 = fc_layer(input=flat,n_inputs=flat.get_shape()[1:4].num_elements(),n_outputs=fc_layer_size)
        fc2 = fc_layer(input=fc1,n_inputs=fc_layer_size,n_outputs=n_classes,use_relu=False) # n_outputs=n_classes
        y_pred = tf.nn.softmax(fc2,name="y_pred")
        y_pred_class = tf.argmax(y_pred, dimension=1)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc2,labels=y_true)
        cost = tf.reduce_mean(cross_entropy)
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
        correct_prediction = tf.equal(y_pred_class, y_true_class)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name="accuracy")
        #print("Graph initialised")

        saver = tf.train.Saver()

        #session run with one kind of label
        with tf.Session() as sess:
            #print("inside session")
            # Run the initializer
            sess.run(tf.global_variables_initializer())
            #print("Initialised")
            x_valid_batch, y_valid_batch = valid_data.next_batch(batch_size)
            feed_dict_val = {X: x_valid_batch, y_true: y_valid_batch}
            val_acc = sess.run(accuracy,feed_dict=feed_dict_val)
            val_loss = sess.run(cost, feed_dict=feed_dict_val)   
            msg = "Pre-training (Epoch {0}) --- Training Accuracy: {1:>6.2%}, Validation Accuracy: {2:>6.2%},  Validation Loss: {3:.3f}"
            print(msg.format(0, 0, val_acc, val_loss)) # , val_loss))
            for i in range(1,n_epochs+1):
                train_data.randomize(sess)
                train_data.batch_index = 0    
                valid_data.randomize(sess)
                valid_data.batch_index = 0
                acc = 0
                for batch in range (n_batches):
                    #if batch % 10 == 0:
                    #    print('Batch', batch, 'of', n_batches, 'done')
                    x_batch, y_true_batch = train_data.next_batch(batch_size)
                    feed_dict_train = {X: x_batch, y_true: y_true_batch}
                    sess.run(optimizer, feed_dict=feed_dict_train)
                    acc += sess.run(accuracy, feed_dict=feed_dict_train)
                acc = acc/n_batches
                if i % display_step == 0: 
                    valid_data.batch_index = 0
                    x_valid_batch, y_valid_batch = valid_data.next_batch(batch_size)
                    feed_dict_val = {X: x_valid_batch, y_true: y_valid_batch}
                    #val_loss = sess.run(cost, feed_dict=feed_dict_val)
                    val_acc = sess.run(accuracy,feed_dict=feed_dict_val)
                    val_loss = sess.run(cost, feed_dict=feed_dict_val)
                    #epoch = int(i / int(data.train.num_examples/batch_size))   
                    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.2%}, Validation Accuracy: {2:>6.2%},  Validation Loss: {3:.3f}"
                    print(msg.format(i, acc, val_acc, val_loss)) # , val_loss))
                if i % saver_step == 0 or val_acc > 0.9:
                    save_path = saver.save(sess, model+"_"+str(i))

print("Done!")                
#print(numTraining)
#print(numValidation)
#print(numTesting)
#print(trainingFaces.shape)
#print(validationFaces.shape)
#print(testingFaces.shape)

Pre-training (Epoch 0) --- Training Accuracy:  0.00%, Validation Accuracy: 75.00%,  Validation Loss: 0.560
Training Epoch 1 --- Training Accuracy: 61.09%, Validation Accuracy: 68.75%,  Validation Loss: 0.639
Training Epoch 2 --- Training Accuracy: 71.88%, Validation Accuracy: 56.25%,  Validation Loss: 0.877
Training Epoch 3 --- Training Accuracy: 75.86%, Validation Accuracy: 68.75%,  Validation Loss: 0.476


In [4]:
# This function will be used to evaluate your submission.

def computeLabels(faceData):
    n, d = faceData.shape
    # Zero arrays for the labels, should be able to do better than this
    estSexLabels = np.zeros(n)
    estAgeLabels = np.zeros(n)
    estExpLabels = np.zeros(n)
    return estSexLabels, estAgeLabels, estExpLabels

estS, estA, estE = computeLabels(validationFaces)
# I'll do stuff with the above to evaluate the accuracy of your methods